In [ ]:
# Installing required packages
%pip install -r requirements.txt

In [ ]:
size_of_heap = 100_000
import random
def gen_random_string(le=12)->str:
    alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789~!@#$%^&*(),.<>/?'"{}[]|-_=+;:`"
    random.seed()
    pwd_len= random.randrange(1,le)
    final_str= ""
    for _ in range(pwd_len):
        char= random.randrange(le)
        final_str += alphabet[char]
    return final_str

In [ ]:
from pympler import asizeof

In [ ]:
from heapitem import HeapItem
import heapq
heap = []
for i in range(size_of_heap):
    heapq.heappush(heap,HeapItem(0.5555,get_random_string()))
    
print(f"sizeof normal heap with heapitem = {asizeof(heap)}")
del heap


In [ ]:
import heapc
heap = heapc.Heap(size_of_heap)
for i in range(size_of_heap):
    heapc.heappush(heap,(0.5555,get_random_string()))
    
print(f"sizeof special heap = {asizeof(heap)}")
del heap


In [6]:
a = ["a","b","c","d"]
s = set()
for n in sorted(a,reverse=True):
    s.add(n)
print(s)

{'d', 'c', 'a', 'b'}
